### Часть первая. Векторизация

In [133]:
# Подключение библиотек
import numpy as np
import pandas as pd
import requests

import torch
import nltk
from sentence_transformers import models, SentenceTransformer
import os
import requests
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
from collections import Counter

from tqdm import tqdm

url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"
stopwords_ru = get_text(url_stopwords_ru).splitlines()
morph = pymorphy2.MorphAnalyzer()


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')
        

def normalize_tokens(tokens):
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def remove_stopwords(tokens, stopwords=None, min_length=4):
    if not stopwords:
        return tokens
    stopwords = set(stopwords)
    tokens = [tok
              for tok in tokens
              if tok not in stopwords and len(tok) >= min_length]
    return tokens


def normalize_rem_stops(words, stopwords=None, normalize=True):
    if normalize:
        words = normalize_tokens(words)
    if stopwords:
        words = remove_stopwords(words, stopwords)
    return words


def prep_text(text):
    words = nltk.tokenize.word_tokenize(text, language="russian")
    words = normalize_rem_stops(words, stopwords=stopwords_ru)
    return words


def get_full_desc(wine_df, idx):
    wine = wine_df.iloc[idx]
    full_desc = f"\nName: {wine['name']}\n\n"
    full_desc += f"Aroma: {wine['aroma']}\n\n"
    full_desc += f"Taste: {wine['taste']}\n\n"
    full_desc += f"Food pairing: {wine['food_pairing']}\n\n"
    full_desc += f"{'='*127}"
    return full_desc

In [160]:
# Вспомогательное
WINE_DATA_PATH = "../data/wine/wines_winestyle.csv"
MAIN_COLS = ["id", "name", "aroma", "taste", "food_pairing", "description"]
DESC_COLS = ["aroma", "taste", "food_pairing"]

MODEL_INFO = "DeepPavlov/rubert-base-cased"

In [161]:
# Чтение данных
wine_df = pd.read_csv(WINE_DATA_PATH, usecols=MAIN_COLS)
wine_df = wine_df.head(10000)

In [162]:
# Обработка пустых значений
wine_df[DESC_COLS] = wine_df[DESC_COLS].fillna("")

In [6]:
# Установка дополнительного nltk-модуля
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /data/user/mikhaillebedev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [163]:
# Токенизация
tokens_col = []
wine_df['full_desc'] = wine_df[DESC_COLS].agg(' '.join, axis=1)
wine_df["tokens"] = [prep_text(text) for text in tqdm(wine_df['full_desc'])]




100%|██████████| 10000/10000 [02:19<00:00, 71.77it/s]


In [164]:
wine_df

,id,name,aroma,taste,food_pairing,description,full_desc,tokens
0,116,"""Chateau Les Jouberts"" Cuvee Prestige, Blaye C...","Вино демонстрирует сложный аромат, сотканный и...","Вино обладает щедрым, ярким вкусом с оттенками...","Вино составит гармоничную пару говядине, утке,...","""Chateau Les Jouberts"" Cuvee Prestige, Blaye C...","Вино демонстрирует сложный аромат, сотканный и...","[вино, демонстрировать, сложный, аромат, сотка..."
1,117,"Ramirez de la Piscina, Crianza, Rioja DOCa, 2015","В глубоком аромате вина ноты вишни, сливы и еж...","Вино демонстрирует насыщенный, сбалансированны...","Вино подходит к блюдам из говядины, телятины, ...","Ramirez de la Piscina, Crianza, Rioja DOCa — к...","В глубоком аромате вина ноты вишни, сливы и еж...","[глубокий, аромат, вино, нота, вишня, слив, еж..."
2,118,"Ramirez de la Piscina, Reserva, Rioja DOCa, 2014",Элегантный аромат вина раскрывается оттенками ...,"Вино демонстрирует округлый, сбалансированный ...","Вино подходит к блюдам из говядины, телятины, ...","Ramirez de la Piscina, Reserva, Rioja DOCa — э...",Элегантный аромат вина раскрывается оттенками ...,"[элегантный, аромат, вино, раскрываться, оттен..."
3,119,"Rafael Cambra, Soplo, Valencia DO, 2014","Вино демонстрирует элегантный, сдержанный аром...","Вкус вина деликатный, мягкий и сбалансированны...","Вино идеально подходит к блюдам из птицы, запе...","Rafael Cambra, Soplo, Valencia DO — красное су...","Вино демонстрирует элегантный, сдержанный аром...","[вино, демонстрировать, элегантный, сдержать, ..."
4,120,"Familia Bastida, ""Drama"" Red Blend, La Mancha ...","Вино демонстрирует глубокий аромат, сотканный ...","Вкус вина сочный, концентрированный, с оттенка...",Вино рекомендуется употреблять с блюдами из мя...,"Красное сухое вино ""Drama"" Red Blend, La Manch...","Вино демонстрирует глубокий аромат, сотканный ...","[вино, демонстрировать, глубокий, аромат, сотк..."
...,...,...,...,...,...,...,...,...
9995,10116,"Moulin des Haies, Muscadet Sevre et Maine Sur ...","Чистый, свежий аромат вина отличается щедрость...","Вино обладает довольно насыщенным вкусом, в ко...",Вино составит хорошее сочетание с морепродукта...,"Moulin des Haies, Muscadet Sevre et Maine Sur ...","Чистый, свежий аромат вина отличается щедрость...","[чистый, свежий, аромат, вино, отличаться, щед..."
9996,10117,"""Greatwall"" Cabernet Sauvignon",Элегантный аромат вина раскрывается оттенками ...,"Вкус вина мягкий, округлый, с фруктово-ягодным...",Вино лучше всего подавать к жареному или тушен...,"""Greatwall"" Cabernet Sauvignon — китайское кра...",Элегантный аромат вина раскрывается оттенками ...,"[элегантный, аромат, вино, раскрываться, оттен..."
9997,10118,"Kumala, ""Reserve"" Pinotage, 2019",Вино привлекает элегантным ароматом с доминиру...,"Вино демонстрирует мягкий, чистый вкус с фрукт...","Вино идеально подходит к трапезе, лучше всего ...","Kumala, ""Reserve"" Pinotage изготовлено из трад...",Вино привлекает элегантным ароматом с доминиру...,"[вино, привлекать, элегантный, аромат, доминир..."
9998,10119,"Echeverria, Cabernet Sauvignon Reserva, 2017","Вино обладает гармоничным, щедрым ароматом, со...","Вино интригует гладким, хорошо структурированн...",Вино составит прекрасную пару со зрелыми сырам...,Cabernet Sauvignon Reserva — одно из элегантны...,"Вино обладает гармоничным, щедрым ароматом, со...","[вино, обладать, гармоничный, щедрый, аромат, ..."


In [165]:
# Отбор токенизированных данных
wine_tokens_df = wine_df[["id", "name", "tokens"]]
wine_tokens_df = wine_tokens_df[wine_tokens_df.tokens.map(len) > 0]
wine_tokens_df

,id,name,tokens
0,116,"""Chateau Les Jouberts"" Cuvee Prestige, Blaye C...","[вино, демонстрировать, сложный, аромат, сотка..."
1,117,"Ramirez de la Piscina, Crianza, Rioja DOCa, 2015","[глубокий, аромат, вино, нота, вишня, слив, еж..."
2,118,"Ramirez de la Piscina, Reserva, Rioja DOCa, 2014","[элегантный, аромат, вино, раскрываться, оттен..."
3,119,"Rafael Cambra, Soplo, Valencia DO, 2014","[вино, демонстрировать, элегантный, сдержать, ..."
4,120,"Familia Bastida, ""Drama"" Red Blend, La Mancha ...","[вино, демонстрировать, глубокий, аромат, сотк..."
...,...,...,...
9995,10116,"Moulin des Haies, Muscadet Sevre et Maine Sur ...","[чистый, свежий, аромат, вино, отличаться, щед..."
9996,10117,"""Greatwall"" Cabernet Sauvignon","[элегантный, аромат, вино, раскрываться, оттен..."
9997,10118,"Kumala, ""Reserve"" Pinotage, 2019","[вино, привлекать, элегантный, аромат, доминир..."
9998,10119,"Echeverria, Cabernet Sauvignon Reserva, 2017","[вино, обладать, гармоничный, щедрый, аромат, ..."


In [143]:
# Создание (загрузка) модели
transformer = models.Transformer(MODEL_INFO)
pooling_module = models.Pooling(transformer.get_word_embedding_dimension())
model = SentenceTransformer(modules=[transformer, pooling_module])

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 84%|████████▎ | 836/1000 [15:36<03:03,  1.12s/it]


In [144]:
# Переход на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

cuda


In [166]:
wine_vectors = [model.encode(token).mean(axis=0).tolist() for token in tqdm(wine_tokens_df["tokens"])]
wine_tokens_df["vector"] = wine_vectors

100%|██████████| 10000/10000 [05:40<00:00, 29.34it/s]


In [268]:
# Сохранение векторизованных данных
wine_vectors_df = wine_tokens_df[["id", "name", "vector"]]
wine_vectors_df.to_csv("../data/wine/wines_vectors_local.csv", index=False)
wine_vectors_df.to_parquet('../data/wine/wines_vectors.parquet', index=False) 
wine_df.to_csv("../data/wine/wines_winestyle_10k.csv", index=False)

### Часть вторая. Рекомендация

In [99]:
# Подключение библиотек
import numpy as np
import pandas as pd
from ast import literal_eval

from sklearn.metrics.pairwise import cosine_similarity

In [263]:
# Вспомогательное
WINE_DATA_PATH = "../data/wine/wines_winestyle.csv"
MAIN_COLS = ["id", "name", "aroma", "taste", "food_pairing", "description"]

WINE_VEC_DATA_PATH = "../data/wine/wines_vectors.parquet"

In [13]:
# Чтение стандартных данных
wine_df = pd.read_csv(WINE_DATA_PATH, usecols=MAIN_COLS)
wine_df

,id,name,aroma,taste,food_pairing,description
0,116,"""Chateau Les Jouberts"" Cuvee Prestige, Blaye C...","Вино демонстрирует сложный аромат, сотканный и...","Вино обладает щедрым, ярким вкусом с оттенками...","Вино составит гармоничную пару говядине, утке,...","""Chateau Les Jouberts"" Cuvee Prestige, Blaye C..."
1,117,"Ramirez de la Piscina, Crianza, Rioja DOCa, 2015","В глубоком аромате вина ноты вишни, сливы и еж...","Вино демонстрирует насыщенный, сбалансированны...","Вино подходит к блюдам из говядины, телятины, ...","Ramirez de la Piscina, Crianza, Rioja DOCa — к..."
2,118,"Ramirez de la Piscina, Reserva, Rioja DOCa, 2014",Элегантный аромат вина раскрывается оттенками ...,"Вино демонстрирует округлый, сбалансированный ...","Вино подходит к блюдам из говядины, телятины, ...","Ramirez de la Piscina, Reserva, Rioja DOCa — э..."
3,119,"Rafael Cambra, Soplo, Valencia DO, 2014","Вино демонстрирует элегантный, сдержанный аром...","Вкус вина деликатный, мягкий и сбалансированны...","Вино идеально подходит к блюдам из птицы, запе...","Rafael Cambra, Soplo, Valencia DO — красное су..."
4,120,"Familia Bastida, ""Drama"" Red Blend, La Mancha ...","Вино демонстрирует глубокий аромат, сотканный ...","Вкус вина сочный, концентрированный, с оттенка...",Вино рекомендуется употреблять с блюдами из мя...,"Красное сухое вино ""Drama"" Red Blend, La Manch..."
...,...,...,...,...,...,...
43196,43490,"Guilbaud Freres, ""Coraline"" Cabernet d'Anjou AOC","Вино обладает тонким, изысканным ароматом с но...","Вкус вина нежный, свежий, питкий, гладкий, сле...","Вино прекрасно в качестве аперитива, в сочетан...","""Coraline"" Cabernet d'Anjou — очень приятное, ..."
43197,43491,"Schlumberger, Gold Trocken, 2012",Игристое вино чарует освежающим ароматом с дом...,"Игристое вино демонстрирует свежий, мягкий вку...",Это изысканное вино рекомендуется подавать к д...,"Schlumberger, Gold Trocken — утонченное белое ..."
43198,43492,"Derbent Wine Company, ""Fanfara"" Rose Brut",Освежающий аромат вина наполнен утонченными но...,"Вкус вина легкий, обладает яркой и живой кисло...","Вино прекрасно сочетается с легкими закусками,...","""Fanfara"" Rose Brut — изысканное игристое вино..."
43199,43493,"""Blue Nun"" 24K Gold Edition",Аромат вина раскрывается приятными и утонченны...,"Вкус вина изысканный, сбалансированный, гармон...","Вино идеально в качестве аперитива, хорошо соч...",В 1857 году Герман Зихель основал небольшой би...


In [267]:
# Чтение векторизованных данных
wine_vectors_df = pd.read_parquet(WINE_VEC_DATA_PATH)
wine_vectors_df

,id,name,vector
0,116,"""Chateau Les Jouberts"" Cuvee Prestige, Blaye C...","[0.19545800983905792, 0.22112557291984558, 0.1..."
1,117,"Ramirez de la Piscina, Crianza, Rioja DOCa, 2015","[0.20425255596637726, 0.21205838024616241, 0.2..."
2,118,"Ramirez de la Piscina, Reserva, Rioja DOCa, 2014","[0.2254984825849533, 0.21893933415412903, 0.19..."
3,119,"Rafael Cambra, Soplo, Valencia DO, 2014","[0.21787668764591217, 0.21199947595596313, 0.1..."
4,120,"Familia Bastida, ""Drama"" Red Blend, La Mancha ...","[0.20609961450099945, 0.2198144942522049, 0.18..."
...,...,...,...
9995,10116,"Moulin des Haies, Muscadet Sevre et Maine Sur ...","[0.1838761270046234, 0.19599491357803345, 0.15..."
9996,10117,"""Greatwall"" Cabernet Sauvignon","[0.24622108042240143, 0.22963517904281616, 0.1..."
9997,10118,"Kumala, ""Reserve"" Pinotage, 2019","[0.19348041713237762, 0.195719376206398, 0.161..."
9998,10119,"Echeverria, Cabernet Sauvignon Reserva, 2017","[0.23446586728096008, 0.20856806635856628, 0.1..."


In [255]:
import itertools

class Recommendator:
    def __init__(self, casual_df, vectors_df):
        self.casual_df = casual_df
        self.vectors_df = vectors_df
        self.vectors = np.concatenate(self.vectors_df["vector"]).reshape((wine_vectors_df.shape[0], -1))
        self.cosim_matrix = None # Для расчёта нужно большое кол-во ресурсов

    def _calculate_cosim_matrix(self):
        self.cosim_matrix = cosine_similarity(vectors, vectors)
    
    def get_jaccard_sim(self, set1, set2): 
        c = set1.intersection(set2)
        return c, float(len(c)) / (len(set1) + len(set2) - len(c))
    
    def recommend_from_description(self, desc, n=10):
        wine_vector = model.encode(prep_text(desc)).mean(axis=0).tolist()
        cosim_vec = cosine_similarity(np.array(wine_vector).reshape(1, -1), self.vectors)[0]
        best_idxs = np.argsort(cosim_vec)[::-1][:n+1]
        best_ids = self.vectors_df.iloc[best_idxs]["id"].tolist()
        
        recommendations = self.casual_df.iloc[pd.Index(self.casual_df["id"]).get_indexer(best_ids)]
        recommendations.reset_index(drop=True, inplace=True)
        return recommendations
    
    def recommend_from_wine(self, wine_identifier, n=10):
        if isinstance(wine_identifier, int):
            idx = self.vectors_df[self.vectors_df["id"] == wine_identifier].index[0]
        elif isinstance(wine_identifier, str):
            idx = self.vectors_df[self.vectors_df["name"] == wine_identifier].index[0]
        
        cosim_vec = cosine_similarity(self.vectors[idx].reshape(1, -1), self.vectors)[0]
        best_idxs = np.argsort(cosim_vec)[::-1][:n+1]
        best_ids = self.vectors_df.iloc[best_idxs]["id"].tolist()
        
        recommendations = self.casual_df.iloc[pd.Index(self.casual_df["id"]).get_indexer(best_ids)]
        recommendations.reset_index(drop=True, inplace=True)
        recommend_tokens = set(tuple(itertools.chain(*recommendations["tokens"])))
        
        query_tokens = set(recommendations["tokens"][0])
        
        random_wines = self.casual_df.iloc[pd.Index(self.casual_df["id"]).get_indexer(np.random.randint(1, 
                                                                                                        self.casual_df.shape[0], 
                                                                                                        n))]
        random_wines.reset_index(drop=True, inplace=True)
        random_tokens = set(tuple(itertools.chain(*random_wines["tokens"])))
        matched_recommended, rec_jacc = self.get_jaccard_sim(query_tokens, recommend_tokens)
        matched_random, rand_jacc = self.get_jaccard_sim(query_tokens, random_tokens)
        print("similarity in recommended: ", "{0:0.2f}".format(rec_jacc))
        print("similarity in random: ", "{0:0.2f}".format(rand_jacc))
        print("matched tokens in recommended: ", matched_recommended)
        print("matched tokens in random: ", matched_random)
        return recommendations
    


In [256]:
# Инициализация рекомендательной системы
sommelier = Recommendator(wine_df, wine_vectors_df)

In [258]:
# Пример работы
# wine_identificator = 119 
wine_identificator = "Rafael Cambra, Soplo, Valencia DO, 2014"

# wine_identificator = 25467 # "Trimbach, Sylvaner, Alsace AOC"
rec = sommelier.recommend_from_wine(wine_identificator)

print(get_full_desc(rec, 0))
print(get_full_desc(rec, 1))

display(rec)

similarity in recommended:  0.21
similarity in random:  0.11
matched tokens in recommended:  {'сбалансированный', 'вкус', 'нота', 'птица', 'элегантный', 'запечь', 'деликатный', 'соткать', 'мясо', 'мягкий', 'демонстрировать', 'фрукт', 'аромат', 'тонкий', 'идеально', 'цвет', 'вино', 'танин', 'подходить', 'спелый', 'приятный', 'блюдо', 'послевкусие', 'сдержать'}
matched tokens in random:  {'идеально', 'сбалансированный', 'мягкий', 'цвет', 'вино', 'элегантный', 'демонстрировать', 'танин', 'спелый', 'приятный', 'блюдо', 'запечь', 'деликатный', 'послевкусие', 'вкус', 'фрукт', 'мясо', 'аромат', 'нота', 'тонкий'}

Name: Rafael Cambra, Soplo, Valencia DO, 2014

Aroma: Вино демонстрирует элегантный, сдержанный аромат, сотканный из нот спелых фруктов и цветов.

Taste: Вкус вина деликатный, мягкий и сбалансированный, с тонкими танинами и приятным послевкусием.

Food pairing: Вино идеально подходит к блюдам из птицы, запеченного мяса и сырам.


Name: Chateau de Beauregard, Fleurie "Colonies de Roch

,id,name,aroma,taste,food_pairing,description,full_desc,tokens
0,119,"Rafael Cambra, Soplo, Valencia DO, 2014","Вино демонстрирует элегантный, сдержанный аром...","Вкус вина деликатный, мягкий и сбалансированны...","Вино идеально подходит к блюдам из птицы, запе...","Rafael Cambra, Soplo, Valencia DO — красное су...","Вино демонстрирует элегантный, сдержанный аром...","[вино, демонстрировать, элегантный, сдержать, ..."
1,6625,"Chateau de Beauregard, Fleurie ""Colonies de Ro...","Вино демонстрирует элегантный аромат, сотканны...","Вкус вина округлый, сбалансированный, с фрукто...","Вино рекомендуют подавать к козьему сыру, блюд...","Chateau de Beauregard, Fleurie ""Colonies de Ro...","Вино демонстрирует элегантный аромат, сотканны...","[вино, демонстрировать, элегантный, аромат, со..."
2,6198,"Undurraga, ""Aliwen"" Cabernet Sauvignon-Syrah R...","Интенсивный, сложный, сочный аромат вина гармо...","Вкус вина хорошо сбалансированный, с мощными, ...",Вино превосходно сочетается с блюдами из красн...,"Aliwen — линейка вин, воплощающая в себе мисти...","Интенсивный, сложный, сочный аромат вина гармо...","[интенсивный, сложный, сочный, аромат, вино, г..."
3,1631,"""Tussock Jumper"" Carmenere","Вино обладает великолепным ароматом, который с...","Вино покоряет мощным, богатым, великолепно сба...",Вино идеально сочетается с блюдами средиземном...,"""Tussock Jumper"" является премиальным винным б...","Вино обладает великолепным ароматом, который с...","[вино, обладать, великолепный, аромат, соткать..."
4,1776,"Derbent Wine Company, ""Di Caspico"" Cabernet Sa...","Вино демонстрирует элегантный, гармоничный аро...","Вкус вина мягкий, округлый, со сбалансированно...",Вино гармонично сочетается с легкими закусками...,"Вина линейки ""Di Caspico"" — гордость каспийско...","Вино демонстрирует элегантный, гармоничный аро...","[вино, демонстрировать, элегантный, гармоничны..."
5,4934,"Fontanafredda, Barolo DOCG, 2014, gif box, 1.5 л","Вино демонстрирует сложный аромат, сотканный и...","Вкус вина полный, округлый, с оттенками вишни,...","Вино великолепно сочетается с дичью, богатыми ...","Fontanafredda, Barolo — яркое, сочное вино с п...","Вино демонстрирует сложный аромат, сотканный и...","[вино, демонстрировать, сложный, аромат, сотка..."
6,4096,"Domaine Saint-Lannes, ""Les Peyrades"", Cotes de...",Вино демонстрирует гармоничный аромат с домини...,"Вкус вина мягкий, сбалансированный, с фруктовы...",Вино отлично сочетается с приготовленным на гр...,"Domaine Saint-Lannes, ""Les Peyrades"" — красное...",Вино демонстрирует гармоничный аромат с домини...,"[вино, демонстрировать, гармоничный, аромат, д..."
7,9549,"""Chatelain Louis"" Rouge Sec","Вино соблазняет приятным, утонченным ароматом ...","Вкус вина округлый, мягкий, свежий, с сочными ...",Вино станет прекрасным сопровождением блюд из ...,"""Chatelain Louis"" Rouge Sec — свежее французск...","Вино соблазняет приятным, утонченным ароматом ...","[вино, соблазнять, приятный, утончить, аромат,..."
8,9470,"""Tierra Santa"" Tinto Seco",Вино отличается ярким и запоминающимся аромато...,"Вино обладает мягким, приятным, питким, фрукто...",Вино создаст отличную гастрономическую пару с ...,"""Tierra Santa"" Tinto Seco — яркое красное сухо...",Вино отличается ярким и запоминающимся аромато...,"[вино, отличаться, яркий, запоминаться, аромат..."
9,2065,"Covinca, ""Vina Oria"" Garnacha, Carinena DO","Вино обладает элегантным, утонченным ароматом ...","Вино демонстрирует выразительный, насыщенный в...","Вино сочетается со многими блюдами, особенно у...","""Vina Oria"" Garnacha — красное сухое вино, соз...","Вино обладает элегантным, утонченным ароматом ...","[вино, обладать, элегантный, утончить, аромат,..."


In [269]:
# rec = sommelier.recommend_from_description("кислое маслянистое горькое")
rec = sommelier.recommend_from_description("спелая вишня и травяной запах")

print(get_full_desc(rec, 0))
print(get_full_desc(rec, 1))

display(rec)


Name: Zonin, Valpolicella DOC

Aroma: Аромат вина наполнен фруктово-ягодными нотами ежевики и спелой вишни.

Taste: Вкус вина свежий, с мягкой текстурой и заманчивыми вишневыми нотками.

Food pairing: Вино хорошо сочетается с неострыми мясными блюдами, макаронами с мясным соусом и зрелыми сырами.


Name: Cusumano, "Jale" Chardonnay, Sicilia DOC, 2018

Aroma: Сложный аромат вина наполнен нотками спелых желтых фруктов (цитрусовых, дыни, папайи), тостов, ванили, сушеного абрикоса и маслянистыми ореховыми оттенками.

Taste: Вкус вина богатый, хорошо сбалансированный и структурированный, с тонкой кислотностью и долгим, гладким, элегантным послевкусием.

Food pairing: Вино прекрасно сочетается со всеми рыбными блюдами, белым мясом под сливочным соусом, грибным ризотто, пастой с пряными травами, молодыми и зрелыми сырами.



,id,name,aroma,taste,food_pairing,description,full_desc,tokens
0,6264,"Zonin, Valpolicella DOC",Аромат вина наполнен фруктово-ягодными нотами ...,"Вкус вина свежий, с мягкой текстурой и заманчи...",Вино хорошо сочетается с неострыми мясными блю...,"Zonin, Valpolicella — красное вино из линейки ...",Аромат вина наполнен фруктово-ягодными нотами ...,"[аромат, вино, наполнить, фруктово-ягодный, но..."
1,1987,"Cusumano, ""Jale"" Chardonnay, Sicilia DOC, 2018",Сложный аромат вина наполнен нотками спелых же...,"Вкус вина богатый, хорошо сбалансированный и с...",Вино прекрасно сочетается со всеми рыбными блю...,"Близ Палермо есть район под названием Джале, с...",Сложный аромат вина наполнен нотками спелых же...,"[сложный, аромат, вино, наполнить, нотка, спел..."
2,5278,"Sophenia, ""Altosur"" Malbec",Аромат вина наполнен насыщенными тонами спелых...,"Вкус вина интенсивный, концентрированный, слад...",Вино прекрасно сочетается со стейком из говяди...,"Название линейки вин ""Altosur"" означает ""Высок...",Аромат вина наполнен насыщенными тонами спелых...,"[аромат, вино, наполнить, насыщенный, спелый, ..."
3,5177,"Гранд Резерв ""Атаман"" Цимлянский Черный",В аромате вина первыми выступают железистые и ...,"Вкус вина понятный, не слишком сложный, с умер...","Вино хорошо сочетается с красным мясом, дичью,...","Гранд Резерв ""Атаман"" Цимлянский Черный — крас...",В аромате вина первыми выступают железистые и ...,"[аромат, вино, выступать, железистый, животное..."
4,3801,"""Marques de Casa Concha"" Cabernet Sauvignon","В букете вина сочные ароматы вишни, смородины ...","Вкус вина концентрированный, с шелковистой тан...","Вино хорошо сочетается с красным мясом, зрелым...","""Marques de Casa Concha"" — один из лучших брен...","В букете вина сочные ароматы вишни, смородины ...","[букет, вино, сочный, аромат, вишня, смородина..."
5,1999,"""Хороший Год"" Бастардо",Утонченный и соблазнительный аромат вина окуты...,"Вино отличается изысканным, гармоничным, отлич...","Вино идеально гармонирует с мясными блюдами, м...","""Хороший Год"" Бастардо — красное сухое вино из...",Утонченный и соблазнительный аромат вина окуты...,"[утончить, соблазнительный, аромат, вино, окут..."
6,2674,"""Легенды Ай-Петри"" Тамянка Южная",Тонкий аромат вина сложен из нот спелых садовы...,"Вкус вина гармоничный, чистый, фруктовый, с но...","Вино рекомендуется сочетать с рыбой, сырами с ...","Линейка столовых вин ""Легенды Ай-Петри"", созда...",Тонкий аромат вина сложен из нот спелых садовы...,"[тонкий, аромат, вино, сложный, нота, спелый, ..."
7,10010,"""Duke of Wellington"" Pinotage, 2017","Элегантный аромат вина окутывает тонами сливы,...","Вкус вина гармоничный, сочный, теплый, с делик...",Вино можно сочетать с блюдами с томатным соусо...,"""Duke of Wellington"" Pinotage является сердцем...","Элегантный аромат вина окутывает тонами сливы,...","[элегантный, аромат, вино, окутывать, слив, ви..."
8,4918,"Domaine de Pignan, Coralie et Floriane, Chatea...",Соблазнительный аромат вина изобилует нотами с...,"Вино отличается многогранным, мощным, изысканн...","Вино отлично сочетается с запеченой уткой, жар...","Domaine de Pignan, Coralie et Floriane, Chatea...",Соблазнительный аромат вина изобилует нотами с...,"[соблазнительный, аромат, вино, изобиловать, н..."
9,752,"Массандра, Портвейн красный ""Ливадия""","Аромат вина комплексный, насыщенный и нежный о...","Вкус вина гармоничный, согревающий, умеренно т...","Вино прекрасно как дижестив, а также хорошо со...","Портвейн красный ""Ливадия"" производят из виног...","Аромат вина комплексный, насыщенный и нежный о...","[аромат, вино, комплексный, насыщенный, нежный..."
